In [1]:
from datasets import load_dataset

dataset1 = load_dataset("TheAIchemist13/hindi_asr_dataset")
dataset2 = load_dataset("TheAIchemist13/hindi_asr_dataset_2")

In [ ]:
# from datasets import concatenate_datasets

# Assuming you have already loaded the datasets
# dataset1 and dataset2 are your loaded DatasetDict objects

# Concatenate the individual datasets
# combined_train = concatenate_datasets([dataset1["train"], dataset2["train"]])
# combined_test = concatenate_datasets([dataset1["test"], dataset2["test"]])

# Create a new DatasetDict
# combined_dataset = {"train": combined_train, "test": combined_test}

In [2]:
from datasets import load_dataset, DatasetDict, concatenate_datasets

combined_dataset1 = DatasetDict()
combined_dataset2 = DatasetDict()
combined_dataset = DatasetDict()
"TheAIchemist13/hindi_asr_dataset_2"

combined_dataset1["train"] = load_dataset("TheAIchemist13/hindi_asr_dataset", "default", split="train", use_auth_token=True)
combined_dataset1["test"] = load_dataset("TheAIchemist13/hindi_asr_dataset", "default", split="test", use_auth_token=True)

combined_dataset2["train"] = load_dataset("TheAIchemist13/hindi_asr_dataset_2", "default", split="train", use_auth_token=True)
combined_dataset2["test"] = load_dataset("TheAIchemist13/hindi_asr_dataset_2", "default", split="test", use_auth_token=True)

combined_dataset["train"] = concatenate_datasets([combined_dataset1["train"], combined_dataset2["train"]])
combined_dataset["test"] = concatenate_datasets([combined_dataset1["test"], combined_dataset2["test"]])
combined_dataset

/usr/local/lib/python3.8/dist-packages/datasets/load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
Using the latest cached version of the dataset since TheAIchemist13/hindi_asr_dataset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/TheAIchemist13___hindi_asr_dataset/default/0.0.0/54a3ac1f465f24d9a296bdeb3cd5ee66218f23ae (last modified on Sat Mar 16 20:59:23 2024).
Using the latest cached version of the dataset since TheAIchemist13/hindi_asr_dataset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/TheAIchemist13___hindi_asr_dataset/default/0.0.0/54a3ac1f465f24d9a296bdeb3cd5ee66218f23ae (last modified on Sat Mar 16 20:59:23 2024).
Using the latest cach

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcriptions'],
        num_rows: 255
    })
    test: Dataset({
        features: ['audio', 'transcriptions'],
        num_rows: 95
    })
})

In [3]:
combined_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcriptions'],
        num_rows: 255
    })
    test: Dataset({
        features: ['audio', 'transcriptions'],
        num_rows: 95
    })
})

In [4]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")

In [5]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v3", language="Hindi", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", language="Hindi", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
print(combined_dataset.keys())

dict_keys(['train', 'test'])


In [8]:
from datasets import Audio

# for split_name in combined_dataset.keys():
#     combined_dataset[split_name] = combined_dataset[split_name].cast_column("audio", Audio(sampling_rate=16000))
combined_dataset = combined_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [9]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcriptions"]).input_ids
    return batch

In [10]:
print(combined_dataset["train"][0])

{'audio': {'path': '1.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, 'transcriptions': 'बीकीपर अक्सर अपनी कॉलोनियों के स्वास्थ्य को सुनिश्चित करने के लिए अमेरिकन फाउलब्रूड परीक्षणों का उपयोग करते हैं।'}


In [11]:
common_voice = combined_dataset.map(prepare_dataset, remove_columns=combined_dataset.column_names["train"], num_proc=5)

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [14]:
import evaluate

metric = evaluate.load("wer")

In [15]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [16]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")

In [17]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [18]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-large-v3",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [19]:
# !pip install accelerate -U
# !pip install transformers -U
import accelerate, transformers

print(transformers.__version__)
accelerate.__version__

4.38.2


'0.28.0'

In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
#     tokenizer=tokenizer
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.8/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [24]:
processor.save_pretrained(training_args.output_dir)

[]

In [30]:
import torch
torch.cuda.empty_cache()

In [31]:
import gc
del variables
gc.collect()

NameError: name 'variables' is not defined

In [32]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 5            |        cudaMalloc retries: 7         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  12010 MiB |  13718 MiB | 246476 MiB | 234466 MiB |\n|       from large pool |  12004 MiB |  13709 MiB | 245489 MiB | 233485 MiB |\n|       from small pool |      6 MiB |      8 MiB |    987 MiB |    980 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |  12010 MiB |  13718 MiB | 246476 MiB | 234466 MiB |\n|       from large pool |  12004 MiB |  13709 MiB |

In [34]:
for m in self.children():
    m.cuda()
    x = m(x)
    m.cpu()
    torch.cuda.empty_cache()
trainer.train()

NameError: name 'self' is not defined

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "hi",
    "model_name": "Whisper Small Hi - Sanchit Gandhi",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="sanchit-gandhi/whisper-small-hi")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

In [ ]:
# !pip install pyunpack

In [ ]:
# !pip -y uninstall zipfile36

In [ ]:
# import zipfile

# with zipfile.Zipfile("", 'r') as zObject:
#     zObject.extractall(path="./darpg_audiofiles")
    
# from pyunpack import Archive

# Archive('./darpg_audiofiles.zip').extractall('./')


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)
generate_kwargs = {"language": "hi"}
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=5,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
    generate_kwargs=generate_kwargs
)

result = pipe("data_files/_6002335182.mp3")
print(result["text"])